# System-Theoretic Methods with pyMOR

## Linear Time-Invariant (LTI) Systems

$$
\begin{align*}
  E \dot{x}(t) & = A x(t) + B u(t), \quad x(0) = 0, \\
  y(t) & = C x(t) + D u(t),
\end{align*}
$$

- $u(t) \in \mathbb{R}^m$ is the input,
- $x(t) \in \mathbb{R}^n$ is the state,
- $y(t) \in \mathbb{R}^p$ is the output.

## Example (Mass-Spring-Damper Chain)

Equations:
$$
\begin{align*}
  m \ddot{q}_1(t)
  + d (2 \dot{q}_1(t) - \dot{q}_2(t))
  + k (2 q_1(t) - q_{2}(t))
  & = u(t), \\
  m \ddot{q}_i(t)
  + d (2 \dot{q}_i(t) - \dot{q}_{i - 1}(t) - \dot{q}_{i + 1}(t))
  + k (2 q_i(t) - q_{i - 1}(t) - q_{i + 1}(t))
  & = 0,
  & i = 2, \ldots, n - 1, \\
  m \ddot{q}_n(t)
  + d (2 \dot{q}_n(t) - \dot{q}_{n - 1}(t))
  + k (2 q_n(t) - q_{n - 1}(t))
  & = 0, \\
  y(t) & = q_n(t)
\end{align*}
$$

Matrix form:
$$
\begin{align*}
  M_{so} \ddot{q}(t)
  + D_{so} \dot{q}(t)
  + K_{so} q(t)
  & = B_{so} u(t), \\
  y(t)
  & = C_{so} q(t)
\end{align*}
$$
where
$$
M_{so} = m I, \quad
D_{so} = d L, \quad
K_{so} = k L, \quad
L =
\begin{bmatrix}
  2 & -1 \\
  -1 & 2 & -1 \\
  & -1 & \ddots & \ddots \\
  & & \ddots & \ddots & -1 \\
  & & & -1 & 2
\end{bmatrix}, \quad
B_{so} = e_1, \quad
C_{so} = e_n^T.
$$

In first-order form:
$$
x(t) =
\begin{bmatrix}
  q(t) \\
  \dot{q}(t)
\end{bmatrix}, \quad
E =
\begin{bmatrix}
  I & 0 \\
  0 & M_{so}
\end{bmatrix}, \quad
A =
\begin{bmatrix}
  0 & I \\
  -K_{so} & -D_{so}
\end{bmatrix}, \quad
B =
\begin{bmatrix}
  0 \\
  B_{so}
\end{bmatrix}, \quad
C =
\begin{bmatrix}
  C_{so} & 0
\end{bmatrix}, \quad
$$

## Building a Full-Order Model

### Matrices

In [ ]:
import numpy as np
import scipy.sparse as sps

In [ ]:
n = 1000
m = 1
d = 10
k = 1
Mso = m * sps.eye(n, format='csc')
L = sps.diags([(n-1)*[-1], n*[2], (n-1)*[-1]], [-1, 0, 1], format='csc')
Dso = d * L
Kso = k * L
Bso = np.zeros((n, 1))
Bso[0, 0] = 1
Cso = np.zeros((1, n))
Cso[0, -1] = 1

In [ ]:
E = sps.block_diag([sps.eye(n), Mso], format='csc')
A = sps.bmat([[None, sps.eye(n)], [-Kso, -Dso]], format='csc')
B = np.vstack([np.zeros((n, 1)), Bso])
C = np.hstack([Cso, np.zeros((1, n))])

### `LTIModel`

In [ ]:
from pymor.models.iosys import LTIModel

In [ ]:
fom = LTIModel.from_matrices(A, B, C, E=E)

In [ ]:
fom

In [ ]:
print(fom)

## Full-Order Model Analysis

In [ ]:
import matplotlib.pyplot as plt
from pymor.core.logger import set_log_levels
set_log_levels({'pymor.algorithms.gram_schmidt.gram_schmidt': 'ERROR'})

### Bode Plot

In [ ]:
w = (5e-4, 5e-2)
_ = fom.transfer_function.bode_plot(w)

### Impulse Response

In [ ]:
from pymor.algorithms.timestepping import ImplicitEulerTimeStepper

In [ ]:
fom = fom.with_(T=2000, time_stepper=ImplicitEulerTimeStepper(2000))

In [ ]:
y = fom.impulse_resp()

In [ ]:
y.shape

In [ ]:
fig, ax = plt.subplots()
_ = ax.plot(np.linspace(0, fom.T, fom.time_stepper.nt + 1), y[:, 0, 0])

### Hankel Singular Values

In [ ]:
hsv = fom.hsv()

In [ ]:
fig, ax = plt.subplots()
_ = ax.semilogy(range(1, len(hsv) + 1), hsv, '.-')
_ = ax.set(
    xlabel='Index $i$',
    ylabel=r'$\sigma_i$',
    title='Hankel singular values'
)

## Balanced Truncation

In [ ]:
from pymor.reductors.bt import BTReductor

In [ ]:
bt = BTReductor(fom)

In [ ]:
rom_bt = bt.reduce(20)

In [ ]:
rom_bt

In [ ]:
print(rom_bt)

### Poles

In [ ]:
fig, ax = plt.subplots()
poles_rom = rom_bt.poles()
_ = ax.plot(poles_rom.real, poles_rom.imag, 'x')
_ = ax.set(
    xlabel='Real',
    ylabel='Imag',
    title='Poles',
)

### Bode Plot

In [ ]:
fig, ax = plt.subplots(2, 1, squeeze=False, figsize=(6, 8), tight_layout=True)
_ = fom.transfer_function.bode_plot(w, ax=ax)
_ = rom_bt.transfer_function.bode_plot(w, ax=ax, linestyle='--')

### Hankel Singular Values

In [ ]:
hsv = fom.hsv()
hsv_bt = rom_bt.hsv()

In [ ]:
fig, ax = plt.subplots()
_ = ax.semilogy(range(1, len(hsv) + 1), hsv, '.-')
_ = ax.semilogy(range(1, len(hsv_bt) + 1), hsv_bt, '.-')
_ = ax.set(
    xlabel='Index $i$',
    ylabel=r'$\sigma_i$',
    title='Hankel singular values'
)

### Error Magnitude Plot

In [ ]:
err_bt = fom - rom_bt

In [ ]:
_ = err_bt.transfer_function.mag_plot(w)

### $\mathcal{H}_2$ Relative Error

In [ ]:
print(err_bt.h2_norm() / fom.h2_norm())

## Iterative Rational Krylov Algorithm (IRKA)

In [ ]:
from pymor.reductors.h2 import IRKAReductor

In [ ]:
irka = ...

In [ ]:
rom_irka = ...

## `SecondOrderModel`

In [ ]:
from pymor.models.iosys import SecondOrderModel

In [ ]:
som = SecondOrderModel.from_matrices(Mso, Dso, Kso, Bso, Cso)

In [ ]:
som

In [ ]:
print(som)

## Second-Order Balanced Truncation

In [ ]:
from pymor.reductors.sobt import SOBTpReductor

In [ ]:
sobtp = ...

In [ ]:
rom_sobtp = ...